In [4]:
import docx
import pandas as pd
import re
import xlrd

book = xlrd.open_workbook('C:\\data-analyst\\临床诊疗术语（治法）2020版.xlsx')
sheet1 = book.sheets()[0]
dic = sheet1.col_values(7)

def extract_docx_text(docFile):
    # 获取文档对象
    document = docx.Document(docFile)
    # 完整的text：
    docx_text = ""
    for para in document.paragraphs:
        docx_text += para.text + '\n'
    return docx_text


if __name__ == '__main__':
    docFile1 = 'C:\\data-analyst\\目录药材.docx'
    docFile2 = 'C:\\data-analyst\\各论(Specific discussion).docx'
    docx_text1 = extract_docx_text(docFile1)
    
    docx_text2 = extract_docx_text(docFile2)

preview = docx_text1.split('\n')
cat_list = []
for x in preview:
    if '《' in x and '》'in x:
        if '附药' in x:
            ix = x.find('附')
            cat_list += [x[:ix]]
        else:
            cat_list += [x] #将所有目录中的药材名字和出处放到一个list; put every material's name and book provenance in a Python list 

def find_all(med, s):
    num = med.find('（《')
    if num == -1:
        return s[:-1]
    else:
        s1 = med[:num]
        s2 = med[num+2:]
        ind = s1.rfind('如')
        if ind == -1:
            return find_all(s2, s)
        else:
            pos = s1[ind+1:]
            if len(pos) > 6:
                return find_all(s2, s)
            else:
                s += pos + ','
                return find_all(s2, s)#将药材的‘应用’里所有的提到的可合成的药物组合起来变成一个大string，用逗号分隔。
            #find every synthesized medicine in '应用'，append them to a string, each med seperated by the commas.
            
def mark(med, dc):
    med = med[:-1]
    mv = med.split('；')
    for x in mv:
        cv = x.split('，')
        for i in cv:
            if i in dic:
                dc += i + ','
    return dc[:-1]

def med_case(mer, med, one, total):
    check = mer.find('【使用注意】')
    for i in range(len(med)):
        if i == 0:
            fortree = med[i].split('\n')
            num1 = fortree[1].find('《')
            fww = re.sub('[\t\ ]', '', fortree[0])
            one.append(fww)
            one.append(fortree[1][num1:])
            index = med[i].find('本品')
            cl = med[i][index:].replace('\n', '')
            clear = re.sub('[a-zA-Z\\t\.\（\） ]','',cl)
            one.append(clear)
        if med[i][:2] == '药性' or med[i][:2] == '性味':
            med1 = med[i][3:-1].replace('\n', '')
            fww2 = re.sub('[\t\ ]', '', med1)
            one.append(fww2)
        elif med[i][:2] == '功效':
            me = med[i][3:-1].replace('\n', '')
            fw2 = re.sub('[\t\ ]', '', me)
            one.append(fw2)
            ch = mark(fw2, '')
            one.append(ch)
        elif med[i][:2] == '应用':
            med2 = med[i][3:-1].replace('\n', '')
            fw = re.sub('[\t\ ]', '', med2)
            one.append(fw)
            syn = find_all(fw, '')
            one.append(syn)
        elif med[i][:4] == '用法用量':
            med4 = med[i][5:-1].replace('\n', '')
            fww3 = re.sub('[\t\ ]', '', med4)
            one.append(fww3)
            if check == -1:
                one.append('')
        elif med[i][:4] == '使用注意':
            med5 = med[i][5:-1].replace('\n', '')
            fww4 = re.sub('[\t\ ]', '', med5)
            one.append(fww4)
        elif med[i][:4] == '现代研究':
            if '附药：' in med[i]:
                x = med[i].find('附药：')
                tp = med[i][5:x].replace('\n', '')
                ts = med[i][x:].replace('\n', '')
                tp2 = re.sub('[\t\ ]', '', tp)
                ind = tp2.find('2.药理作用')
                fina = tp2[ind+6:]
                ts2 = re.sub('[\t\ ]', '', ts)
                one.append(fina)
                one.append(ts2)
                total.append(one)
                break
            else:
                tp2 = med[i][5:].replace('\n', '')
                tp3 = re.sub('[\t\ ]', '', tp2)
                ind = tp3.find('2.药理作用')
                fina2 = tp3[ind+6:]
                one.append(fina2)
                one.append('')
                total.append(one)
                break#将每一个药材信息放到一个list里
                
                
def spl_med(tr):
    meds = ''
    medt = []
    for x in range(len(tr)):
        if tr[x] in cat_list:
            a = meds.rfind('\n')
            meds = meds[:a]
            medt.append(meds)
            meds = tr[x-1] + '\n' + tr[x]
        if x == len(tr)-1:
            meds += '\n' + tr[x]
            medt.append(meds)
            break
        else:
            meds += '\n' + tr[x] #把整个章节通过各种药材分段并放到一个list里,split the whole chapter by medicine, put each
            #medicine info into a list 
    return medt

章节 = docx_text2.split('章\t')
章节 = 章节[1:]



total = []
for chapter in 章节:
    one = []
    num1 = chapter.find('\n')
    one.append(chapter[:num1])
    index1 = chapter.rfind('。')
    chapter = chapter[:index1+1]
    小节 = chapter.split('节\t')
    if len(小节) == 1:
        one.append('')
        tr = 小节[0].split('\n')
        medt = spl_med(tr)[1:]
        for y in range(len(medt)):
            mer = medt[y]
            med = mer.split('【')
            med_case(mer, med, one, total) 
            one = one[:2]
    else:
        小节 = 小节[1:]
        for c in 小节:
            one = one[:1]
            index = c.rfind('。')
            top = c[:index+1]
            findj = top.find('\n')
            one.append(top[:findj])
            tr2 = top.split('\n')
            medt = spl_med(tr2)[1:]
            for y in range(len(medt)):
                mer = medt[y]
                med = mer.split('【')
                med_case(mer, med, one, total)
                one = one[:2]
                
    

column=['一级标签','二级标签','药材名','出处','简介','药性','功效', '治法', '应用', '合成', '用法用量', '使用注意', '现代研究', '附药'] 
test=pd.DataFrame(total, columns=column)
test.to_csv('C:\\Users\\Lion Yang\\Desktop\\Python2\\中药project\\中药学模板.csv' ,encoding= 'utf_8_sig')                 

In [ ]:
import pdfplumber
import numpy as np
import pandas as pd
pdf = pdfplumber.open("C:\\data-analyst\\中药学.pdf")
needed_pages = pdf.pages[147:190]
preview = pdf.pages[20:61]

a = []
s = ''
for i in range(len(needed_pages)):
    a = needed_pages[i].extract_text()
    s = s + a + '\n' #将所有页的内容合并成一整个string, merge the text in each page to a whole string

章节 = s.split('章\u3000')
章节.remove('第八')#将每一个章分开，split every chapter    

f = ''    
for y in range(len(preview)):
    a = preview[y].extract_text()
    f = f + a + '\n'#在目录中把所有的药物出处和拼音名字放到一个list里, put every medicine's Pingyin name and origin in a list


b = f.split('\n')
cat_list = []
for x in b:
    if '《' in x and '》':
        cat_list += [x]

def med_case(mer, med, one, total):
    check = mer.find('【使用注意】')
    for i in range(len(med)):
        if i == 0:
            fortree = med[i].split('\n')
            num1 = fortree[1].find('《')
            one.append(fortree[0])
            one.append(fortree[1][num1:])
            index = med[i].find('本品')
            cl = med[i][index:].replace('\n', '')
            one.append(cl)
        if med[i][:2] == '药性':
            med1 = med[i][3:-1].replace('\n', '')
            one.append(med1)
        elif med[i][:2] == '功效':
            med2 = med[i][3:-1].replace('\n', '')
            one.append(med2)
        elif med[i][:2] == '应用':
            med3 = med[i][3:-1].replace('\n', '')
            one.append(med3)
        elif med[i][:4] == '用法用量':
            med4 = med[i][5:-1].replace('\n', '')
            one.append(med4)
            if check == -1:
                one.append('')
        elif med[i][:4] == '使用注意':
            med5 = med[i][5:-1].replace('\n', '')
            one.append(med5)
        elif med[i][:4] == '现代研究':
            if '附药：' in med[i]:
                x = med[i].find('附药：')
                tp = med[i][5:x].replace('\n', '')
                one.append(tp)
                total.append(one)
            else:
                tp2 = med[i][5:].replace('\n', '')
                one.append(tp2)
                total.append(one) #将每一个药材信息放到一个list里
                
def spl_med(tr):
    meds = ''
    medt = []
    for x in range(len(tr)):
        if tr[x] in cat_list:
            a = meds.rfind('\n')
            meds = meds[:a]
            medt.append(meds)
            meds = tr[x-1] + '\n' + tr[x]
        if x == len(tr)-1:
            meds += '\n' + tr[x]
            medt.append(meds)
            break
        else:
            meds += '\n' + tr[x] #把整个章节通过各种药材分段并放到一个list里,split the whole chapter by medicine, put each
            #medicine info into a list 
    return medt
                         
        
total = []
for chapter in 章节:
    one = []
    num1 = chapter.find('\n')
    one.append(chapter[:num1])
    index1 = chapter.rfind('第')
    chapter = chapter[:index1-1]
    小节 = chapter.split('节\u3000')
    if len(小节) == 1:
        one.append('')
        tr = 小节[0].split('\n')
        medt = spl_med(tr)[1:]
        for y in range(len(medt)):
            mer = medt[y]
            med = mer.split('【')
            med_case(mer, med, one, total) 
            one = one[:2]
    else:
        小节 = 小节[1:]
        for i in range(len(小节)):
            one = one[:1]
            index = 小节[i].rfind('第')
            top = 小节[i][:index]
            findj = top.find('\n')
            one.append(top[:findj])
            tr2 = top.split('\n')
            medt = spl_med(tr2)[1:]
            for y in range(len(medt)):
                mer = medt[y]
                med = mer.split('【')
                med_case(mer, med, one, total)
                one = one[:2]
            

column=['一级标签','二级标签','药材名','出处','简介','药性','功效', '应用', '用法用量', '使用注意', '现代研究'] 
test=pd.DataFrame(total, columns=column)
test.to_csv('C:\\Users\\Lion Yang\\Desktop\\Python2\\中药project\\中药学模板.csv' ,encoding= 'utf_8_sig') 